In [83]:
# =========================
# 1. IMPORT LIBRARY
# =========================
import pandas as pd
import folium
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from folium.elements import MacroElement
from jinja2 import Template

# =========================
# 2. LOAD DATA
# =========================
df = pd.read_csv("/content/Penyebaran Bencana Alam di Indonesia.csv")

# =========================
# 3. PRA-PROSES DATA
# =========================
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['long'] = pd.to_numeric(df['long'], errors='coerce')

df = df[df['status'] == 'confirmed']
df = df.dropna(subset=['lat', 'long'])

print("Jumlah data valid:", len(df))

# =========================
# 4. AI - CLUSTERING (KMeans)
# =========================
X = df[['lat', 'long']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=4, random_state=42)
df['cluster'] = kmeans.fit_predict(X_scaled)

# =========================
# 5. INISIALISASI PETA
# =========================
m = folium.Map(
    location=[df['lat'].mean(), df['long'].mean()],
    zoom_start=5,
    tiles=None
)

# =========================
# 6. BASEMAP
# =========================

# OpenStreetMap
folium.TileLayer(
    tiles="OpenStreetMap",
    name="OpenStreetMap"
).add_to(m)

# Satelit
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name="Satelit"
).add_to(m)

# TOPOGRAFI (ESRI World Topographic Map)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}",
    attr="Esri, USGS, NOAA",
    name="Topografi"
).add_to(m)

# Carto Light
folium.TileLayer(
    tiles="CartoDB positron",
    name="Carto Light"
).add_to(m)

# =========================
# 7. STYLE SIMBOLOGI BENCANA
# =========================
style = {
    'flood':      ('blue', 'tint'),
    'landslide':  ('orange', 'exclamation-triangle'),
    'earthquake': ('red', 'home'),
    'fire':       ('darkred', 'fire'),
    'wind':       ('green', 'cloud'),
    'volcano':    ('black', 'warning')
}

# =========================
# 8. FEATURE GROUP (CHECKBOX)
# =========================
feature_groups = {}

for disaster in df['disaster_type'].unique():
    feature_groups[disaster] = folium.FeatureGroup(
        name=disaster.title()
    ).add_to(m)

# =========================
# 9. TAMBAH MARKER
# =========================
for _, row in df.iterrows():
    color, icon = style.get(row['disaster_type'], ('gray', 'info-sign'))

    folium.Marker(
        location=[row['lat'], row['long']],
        icon=folium.Icon(
            color=color,
            icon=icon,
            prefix='fa'
        ),
        popup=f"""
        <b>Jenis Bencana:</b> {row['disaster_type'].title()}<br>
        <b>Provinsi:</b> {row['Provinsi']}<br>
        <b>Kabupaten:</b> {row['Kabupaten']}<br>
        <b>Kecamatan:</b> {row['Kecamatan']}<br>
        <b>Tanggal:</b> {row['date']}<br>
        <b>Cluster:</b> {row['cluster']}
        """
    ).add_to(feature_groups[row['disaster_type']])

# =========================
# 10. LAYER CONTROL
# =========================
folium.LayerControl(collapsed=False).add_to(m)

# =========================
# 11. LEGENDA (POJOK KIRI BAWAH)
# =========================
legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed;
    bottom: 40px;
    left: 20px;
    width: 230px;
    background-color: white;
    z-index:9999;
    padding: 12px;
    border-radius: 8px;
    box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
    font-size: 14px;
">
<b>Legenda Bencana</b><br><br>
<i class="fa fa-tint" style="color:blue"></i> Banjir<br>
<i class="fa fa-exclamation-triangle" style="color:orange"></i> Longsor<br>
<i class="fa fa-home" style="color:red"></i> Gempa Bumi<br>
<i class="fa fa-fire" style="color:darkred"></i> Kebakaran<br>
<i class="fa fa-cloud" style="color:green"></i> Angin Kencang<br>
<i class="fa fa-warning" style="color:black"></i> Gunung Api
</div>
{% endmacro %}
"""

legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# =========================
# 12. SIMPAN KE HTML
# =========================
m.save("map.html")

print("Peta berhasil dibuat: map.html")


Jumlah data valid: 4235
Peta berhasil dibuat: map.html
